In [1]:
import os
import cv2
import lmdb
import caffe
import random
import numpy as np
from make_lmdb import *

In [2]:
lmdb_name = "/home/nvme/LUSC/valid-lmdb"

N = get_num_records(lmdb_name)
print("# records", N)

# records 21704


In [40]:
cnt = 0
env = lmdb.open(lmdb_name, readonly=True)
with env.begin() as txn:
    cursor = txn.cursor()
    for key, value in cursor:
        if cnt < 10:
            cnt += 1
            print(key)
            key = key.decode('ascii')
            key = key.split('_', 1)[1]
            
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(value)
            flat_x = np.fromstring(datum.data, dtype=np.uint8)
            x = flat_x.reshape(datum.channels, datum.height, datum.width)
            x = np.transpose(x, (1,2,0))
            y = datum.label
            print(x.shape, y)
            
            cv2.imwrite(os.path.basename(key), x)
        else:
            cursor.close()
            break

b'00000000_/home/nvme/LUSC/valid/positive/TCGA-34-2604_i13_x15785_y10671_90.jpg'
(224, 224, 3) 1
b'00000001_/home/nvme/LUSC/valid/positive/TCGA-34-5928_i7_x15019_y25766_90.jpg'
(224, 224, 3) 1
b'00000002_/home/nvme/LUSC/valid/positive/TCGA-34-5928_i7_x14691_y25814_270.jpg'
(224, 224, 3) 1
b'00000003_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i50_x4757_y2432_90.jpg'
(224, 224, 3) 8
b'00000004_/home/nvme/LUSC/valid/positive/TCGA-34-2605_i30_x17224_y18123_180.jpg'
(224, 224, 3) 1
b'00000005_/home/nvme/LUSC/valid/negative/TCGA-33-4547_i0_x8468_y5495.jpg'
(224, 224, 3) 0
b'00000006_/home/nvme/LUSC/valid/negative/TCGA-18-4086_i4_x6515_y4929_180.jpg'
(224, 224, 3) 0
b'00000007_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i19_x13047_y5082_180.jpg'
(224, 224, 3) 8
b'00000008_/home/nvme/LUSC/valid/negative/TCGA-33-4547_i2_x5771_y3855_270.jpg'
(224, 224, 3) 0
b'00000009_/home/nvme/LUSC/valid/positive/TCGA-34-2605_i28_x18063_y21909_90.jpg'
(224, 224, 3) 8


/home/lukawa/.conda/envs/algo-work/lib/python3.5/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


In [3]:
add_data_name = "/home/nvme/LUSC/add.txt"

make_lmdb(add_data_name, lmdb_name, append=True, size=224)

Existed num of samples:  21704
2019-03-11 14:25:31.957118  -->  0


In [29]:
N = get_num_records(lmdb_name)
print("# records", N)

# records 21715


In [17]:
keys = ['00000003_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i50_x4757_y2432_90.jpg', 
        '00000007_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i19_x13047_y5082_180.jpg', 
        '00000009_/home/nvme/LUSC/valid/positive/TCGA-34-2605_i28_x18063_y21909_90.jpg']
cat = 'cat.jpg'

size = 224
map_size = N * 3 * size * size * 3  # last 3 for redundancy purposes
env = lmdb.open(lmdb_name, map_size=map_size)
with env.begin(write=True) as txn:
    for key in keys:
        str_id = key.encode('ascii')
        
        # delete key
        status = txn.delete(str_id)
        if not status:
            print("Does not exist:", key)
            continue

        # put key, value
        img = read_image(cat, size)
        datum = caffe.proto.caffe_pb2.Datum()
        datum.channels = 3
        datum.height = size
        datum.width = size
        datum.data = img.tobytes()  # or .tostring() if numpy < 1.9
        datum.label = 5

        txn.put(str_id, datum.SerializeToString())

Does not exist: 00000003_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i50_x4757_y2432_90.jpg


In [39]:
keys = [b'00000003_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i50_x4757_y2432_90.jpg', 
        b'00000007_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i19_x13047_y5082_180.jpg', 
        b'00000009_/home/nvme/LUSC/valid/positive/TCGA-34-2605_i28_x18063_y21909_90.jpg']
keyi = 0

cnt = 0
map_size = N * 3 * size * size * 3  # last 3 for redundancy purposes
env = lmdb.open(lmdb_name, map_size=map_size)
with env.begin(write=True) as txn:
    cursor = txn.cursor()
    for key, value in cursor:
        if cnt % 10000 == 0:
            print("processed ", cnt)
        if cnt < 10:
            cnt += 1
            print(key)
            if key == keys[keyi]:
                print("key matched")
                if keyi == len(keys):
                    break
                else:
                    keyi += 1
            
                datum = caffe.proto.caffe_pb2.Datum()
                datum.ParseFromString(value)
                
                datum_new = caffe.proto.caffe_pb2.Datum()
                datum_new.channels = 3
                datum_new.height = size
                datum_new.width = size
                datum_new.data = datum.data
                datum_new.label = 8
                
                cursor.replace(key, datum_new.SerializePartialToString())
#                 txn.put(key, datum_new.SerializePartialToString(), dupdata=False, overwrite=True)
        else:
            cursor.close()
            break
    cursor.close()

processed  0
b'00000000_/home/nvme/LUSC/valid/positive/TCGA-34-2604_i13_x15785_y10671_90.jpg'
b'00000001_/home/nvme/LUSC/valid/positive/TCGA-34-5928_i7_x15019_y25766_90.jpg'
b'00000002_/home/nvme/LUSC/valid/positive/TCGA-34-5928_i7_x14691_y25814_270.jpg'
b'00000003_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i50_x4757_y2432_90.jpg'
key matched
b'00000004_/home/nvme/LUSC/valid/positive/TCGA-34-2605_i30_x17224_y18123_180.jpg'
b'00000005_/home/nvme/LUSC/valid/negative/TCGA-33-4547_i0_x8468_y5495.jpg'
b'00000006_/home/nvme/LUSC/valid/negative/TCGA-18-4086_i4_x6515_y4929_180.jpg'
b'00000007_/home/nvme/LUSC/valid/positive/TCGA-22-4599_i19_x13047_y5082_180.jpg'
key matched
b'00000008_/home/nvme/LUSC/valid/negative/TCGA-33-4547_i2_x5771_y3855_270.jpg'
b'00000009_/home/nvme/LUSC/valid/positive/TCGA-34-2605_i28_x18063_y21909_90.jpg'
key matched
